In [17]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('C:/Users/RTCNO00041/Downloads/catanstats.csv')
print(df)

     gameNum  player  points   me  2  3  4  5   6   7  ...  Unnamed: 26  \
0          1       1       5  NaN  1  3  5  8   7  10  ...            O   
1          1       2       9  1.0  1  3  5  8   7  10  ...            O   
2          1       3      10  NaN  1  3  5  8   7  10  ...            C   
3          1       4       5  NaN  1  3  5  8   7  10  ...            S   
4          2       1      10  NaN  1  6  3  9  10   8  ...           2L   
..       ...     ...     ...  ... .. .. .. ..  ..  ..  ...          ...   
195       49       4       5  NaN  0  5  3  6  13  13  ...            S   
196       50       1       5  NaN  1  7  4  5   7  17  ...            C   
197       50       2      10  1.0  1  7  4  5   7  17  ...            C   
198       50       3       4  NaN  1  7  4  5   7  17  ...            S   
199       50       4       8  NaN  1  7  4  5   7  17  ...            C   

     production  tradeGain  robberCardsGain  totalGain  tradeLoss  \
0            38          5    

In [3]:
# Lets clean the data, we have two many columns, S1R1 = Settlement1, Resource1
df['S1R1'] = (df['settlement1'].astype(str) + df['Unnamed: 16'].astype(str)).str.replace('\n', '')
df['S1R2'] = (df['Unnamed: 17'].astype(str) + df['Unnamed: 18'].astype(str)).str.replace('\n', '')
df['S1R3'] = (df['Unnamed: 19'].astype(str) + df['Unnamed: 20'].astype(str)).str.replace('\n', '')
df['S2R1'] = (df['settlement2'].astype(str) + df['Unnamed: 22'].astype(str)).str.replace('\n', '')
df['S2R2'] = (df['Unnamed: 23'].astype(str) + df['Unnamed: 24'].astype(str)).str.replace('\n', '')
df['S2R3'] = (df['Unnamed: 25'].astype(str) + df['Unnamed: 26'].astype(str)).str.replace('\n', '')

# Get rid of old columns
# df.drop(columns=['settlement1','settlement2','Unnamed: 17','Unnamed: 18','Unnamed: 19','Unnamed: 20','Unnamed: 22','Unnamed: 23','Unnamed: 24','Unnamed: 25','Unnamed: 26'])


# I want to change the naming
# L lumber -> W wood
# C Clay -> B Brick
# O Ore -> R Rock
# W Wheat -> WH
# S Sheep

df[['S1R1',"S1R2","S1R3","S2R1","S2R2","S2R3"]] = df[['S1R1',"S1R2","S1R3","S2R1","S2R2","S2R3"]].replace({'L': 'W', 'C': 'B', 'O': 'R', 'W': 'WH', 'S': 'Sheep'}).astype(str)
print(df.describe)



<bound method NDFrame.describe of      gameNum  player  points   me  2  3  4  5   6   7  ...  robberCardsLoss  \
0          1       1       5  NaN  1  3  5  8   7  10  ...                2   
1          1       2       9  1.0  1  3  5  8   7  10  ...                1   
2          1       3      10  NaN  1  3  5  8   7  10  ...                4   
3          1       4       5  NaN  1  3  5  8   7  10  ...                6   
4          2       1      10  NaN  1  6  3  9  10   8  ...               10   
..       ...     ...     ...  ... .. .. .. ..  ..  ..  ...              ...   
195       49       4       5  NaN  0  5  3  6  13  13  ...                4   
196       50       1       5  NaN  1  7  4  5   7  17  ...                6   
197       50       2      10  1.0  1  7  4  5   7  17  ...                6   
198       50       3       4  NaN  1  7  4  5   7  17  ...                1   
199       50       4       8  NaN  1  7  4  5   7  17  ...               12   

     tribute  tot

In [4]:
# We have no player info, except for that "Me" column. Lets see if this person won atleast 25% of the time. We ideally want players of equal caliber. Note that 50 sample size is low to achieve a true skill rating
Games_won = len(df[(df['points'] == 10) & (df['me'] == 1)])
Games_played = len(df[df['me'] == 1])
print(Games_won)
print(Games_played)
print(Games_won / Games_played)

# He won almost half the games, which is pretty high

23
50
0.46


In [12]:
# For all games who won, what was there settlement resource combinations?
winners = df[df['points'] == 10]

# First, lets ignore the numbers for the most part and look at the resource
df[['S1R1',"S1R2","S1R3","S2R1","S2R2","S2R3"]].apply(tuple, axis=1).value_counts()

# We have too many combinations of starting positions.
# We can look at the number but that doesnt depict strategy
# We can look at the suit but that doesnt give info on the numbers
# What we can do is group the numbers into categories (high, medium, low) to be able to look at fewer combinations


(6L, 3C, 11C, 9L, 10W, 11O)    1
(5O, 8C, 10S, 6W, 9W, 3W)      1
(5L, 8S, 10W, 8C, 10L, 02W)    1
(6C, 9W, 10L, 8O, 4L, 02C)     1
(5L, 9O, 10S, 6C, 3S, 12W)     1
                              ..
(5O, 8C, 10C, 6S, 3W, 11L)     1
(8C, 4W, 02S, 5L, 6S, 12S)     1
(6L, 9S, 3W, 5L, 9C, 10O)      1
(5O, 9W, 10W, 3O, 10L, 11C)    1
(3W, 9O, 10S, 3L, 4S, 5C)      1
Length: 200, dtype: int64

In [18]:
# Define a custom function to map strings to categories
def map_to_category(value):
    # Use regular expressions to extract the numeric part from the string
    numeric_part_match = re.search(r'^(\d+)', value)
    
    if numeric_part_match:
        numeric_part = int(numeric_part_match.group(1))
        if numeric_part in [6, 8]:
            return 'High'
        elif numeric_part in [4, 5, 9, 10]:
            return 'Medium'
        elif numeric_part in [1, 2, 11, 3]:
            return 'Low'
    
    return 'Unknown'  # Handle unexpected values if any

columns_to_map = ['S1R1',"S1R2","S1R3","S2R1","S2R2","S2R3"]

for column in columns_to_map:
    df[column + '_amount'] = df[column].apply(map_to_category)

df[['S1R1_amount',"S1R2_amount","S1R3_amount","S2R1_amount","S2R2_amount","S2R3_amount"]].apply(tuple, axis=1).value_counts().head(10)

# Made a mistake and forgot about the resource, but it is interesting to see that the top 2 winning combinations are significantly higher
# We can see an obvious pattern that they only have one Low number between both settlements. 
# That is a bit daunting as it means theres a high weight on map availability as a pose to skill.

# Conclusion, try not to have two numbers in the list [1, 2, 11, 3]
# Sidenote: sample size too small to be sure


(Medium, High, Medium, Medium, Medium, Low)    21
(Medium, High, Medium, High, Medium, Low)      18
(High, Medium, Low, Medium, Medium, Low)       11
(Medium, High, Medium, High, Low, Low)         10
(Medium, Medium, Medium, High, Medium, Low)    10
(High, Medium, Low, Medium, High, Medium)       9
(High, Medium, Low, High, Medium, Low)          8
(Medium, High, Low, Medium, Medium, Low)        6
(Medium, Medium, Low, High, Medium, Low)        5
(Medium, Medium, Medium, High, Low, Low)        5
dtype: int64